In [2]:
%pylab inline
import os
import numpy as np
import pandas as pd
from scipy.misc import imread
from sklearn.metrics import accuracy_score

import tensorflow as tf
import keras

Populating the interactive namespace from numpy and matplotlib


/home/sayak/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['seed']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"
Using TensorFlow backend.


In [3]:
seed = 128
rng = np.random.RandomState(seed)

In [4]:
import os
root_dir = "/media/sayak/Local Disk/TF Shizzle"
data_dir = "/media/sayak/Local Disk/TF Shizzle/data"
sub_dir = "/media/sayak/Local Disk/TF Shizzle/sub"

# check for existence
print os.path.exists(root_dir)
print os.path.exists(data_dir)
print os.path.exists(sub_dir)

True
True
True


In [5]:
train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
test = pd.read_csv(os.path.join(data_dir, 'Test.csv'))
train.head()


,filename,label
0,0.png,4
1,1.png,9
2,2.png,1
3,3.png,7
4,4.png,3


In [6]:
img_name = rng.choice(train.filename)
filepath = os.path.join(data_dir, 'Images', 'train', img_name)

img = imread(filepath, flatten=True)

In [18]:
temp = []
for img_name in train.filename:
    image_path = os.path.join(data_dir, 'Images', 'train', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
train_x = np.stack(temp)

train_x /= 255.0
#train_x = train_x.reshape(-1, 784).astype('float32')
train_x = train_x.reshape(train_x.shape[0], 1, 28, 28).astype('float32')

temp = []
for img_name in test.filename:
    image_path = os.path.join(data_dir, 'Images', 'test', img_name)
    img = imread(image_path, flatten=True)
    img = img.astype('float32')
    temp.append(img)
    
test_x = np.stack(temp)

test_x /= 255.0
#test_x = test_x.reshape(-1, 784).astype('float32')
test_x = test_x.reshape(test_x.shape[0], 1, 28, 28).astype('float32')

#train_x = train_x.astype('float32')
#test_x = test_x.astype('float32')

In [20]:
test_x.shape
train_x.shape
#val_x.shape

(49000, 1, 28, 28)

In [22]:
train_y = keras.utils.np_utils.to_categorical(train.label.values)

In [11]:
split_size = int(train_x.shape[0]*0.7)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = train_y[:split_size], train_y[split_size:]

In [9]:
train.label.ix[split_size:]


34300    3
34301    1
34302    6
34303    8
34304    3
34305    8
34306    8
34307    9
34308    3
34309    8
34310    4
34311    6
34312    6
34313    3
34314    6
34315    7
34316    5
34317    3
34318    0
34319    3
34320    9
34321    3
34322    8
34323    8
34324    7
34325    4
34326    3
34327    8
34328    6
34329    5
        ..
48970    7
48971    5
48972    0
48973    1
48974    4
48975    1
48976    7
48977    5
48978    6
48979    5
48980    6
48981    3
48982    5
48983    5
48984    9
48985    2
48986    9
48987    0
48988    0
48989    7
48990    0
48991    1
48992    1
48993    6
48994    9
48995    2
48996    4
48997    9
48998    3
48999    0
Name: label, dtype: int64

In [23]:
train_x.shape

(49000, 1, 28, 28)

In [29]:

#train_x = train_x.reshape(train_x.shape[0], img_cols, img_rows, 1)
#test_x = test_x.reshape(test_x.shape[0], img_cols, img_rows, 1)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

#input_num_units = 784
#hidden_num_units = 50
#output_num_units = 10

epochs = 5
batch_size = 128


model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

trained_model = model.fit(train_x, train_y, nb_epoch=epochs, batch_size=batch_size, validation_data=(val_x, val_y))

Train on 49000 samples, validate on 14700 samples
Epoch 1/5
49000/49000 [==============================] - 212s - loss: 0.3611 - acc: 0.8868 - val_loss: 0.0987 - val_acc: 0.9701
Epoch 2/5
49000/49000 [==============================] - 205s - loss: 0.0967 - acc: 0.9696 - val_loss: 0.0546 - val_acc: 0.9838
Epoch 3/5
49000/49000 [==============================] - 206s - loss: 0.0688 - acc: 0.9785 - val_loss: 0.0469 - val_acc: 0.9854
Epoch 4/5
49000/49000 [==============================] - 211s - loss: 0.0550 - acc: 0.9826 - val_loss: 0.0288 - val_acc: 0.9917
Epoch 5/5
49000/49000 [==============================] - 226s - loss: 0.0462 - acc: 0.9853 - val_loss: 0.0281 - val_acc: 0.9912


In [30]:
pred = model.predict_classes(test_x)

img_name = rng.choice(test.filename)
filepath = os.path.join(data_dir, 'Images', 'test', img_name)

img = imread(filepath, flatten=True)

test_index = int(img_name.split('.')[0]) - train.shape[0]

print "Prediction is: ", pred[test_index]

20992/21000 [============================>.] - ETA: 0sPrediction is:  0


In [31]:
sample_submission = pd.read_csv(os.path.join(data_dir, 'Sample_Submission.csv'))

sample_submission.filename = test.filename

sample_submission.label = pred

sample_submission.to_csv(os.path.join(sub_dir, 'sub01.csv'), index=False)